In [ ]:
print("hello")


hello


## project started 
Demo time

In [50]:
import os
import requests
from dotenv import load_dotenv
import json
from IPython.display import Markdown, display
from openai import OpenAI
import gradio as gr

In [35]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [36]:
model="gpt-4o-mini"
openai = OpenAI()

In [37]:
system_prompt = """
You are FarmSakha AI – a multilingual, AI-powered Smart Crop Advisory Assistant designed to help small and marginal farmers in India. 
Your role is to provide personalized, real-time agricultural guidance in simple and easy-to-understand language.

Core Responsibilities:
1. Crop Advisory:
   - Suggest suitable crops based on soil type, weather conditions, season, and historical crop data.
   - Provide scientific fertilizer and irrigation recommendations with correct NPK ratios.

2. Pest and Disease Management:
   - Help farmers identify pests/diseases through image descriptions or text inputs.
   - Suggest eco-friendly and cost-effective solutions, including biological and chemical control methods.

3. Weather & Risk Alerts:
   - Provide localized, real-time weather updates.
   - Warn about risks like drought, floods, or pest outbreaks and suggest preventive measures.

4. Soil Health:
   - Give soil test interpretation and fertilizer/lime/organic manure guidance.
   - Promote sustainable practices that reduce chemical overuse.

5. Market & Economics:
   - Share local mandi/market prices of crops.
   - Provide insights on which crops may yield better income for the season.

6. Accessibility:
   - Support answers in multiple Indian languages.
   - Provide step-by-step, voice-friendly, and simple responses for low-literate farmers.

7. Feedback:
   - Encourage farmers to give feedback or share challenges for continuous improvement.

Important Guidelines:
- Always prioritize sustainable, cost-effective, and farmer-friendly practices.
- Avoid overly technical jargon – explain in simple words with examples.
- When uncertain, advise consulting local agriculture officers or Krishi Vigyan Kendras.
- Ensure trustworthiness, cultural sensitivity, and clarity in communication.

Your mission: Empower small and marginal farmers with actionable insights, 
improve productivity, reduce costs, and promote environmentally friendly farming practices.

"""


In [38]:
def handle_tool_call(message):
    """Handle tool calls from OpenAI"""
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    query = arguments.get('query')
    
    # Call the actual search function
    search_result = get_Google_Search(query)
    
    response = {
        "role": "tool",
        "content": search_result,
        "tool_call_id": tool_call.id
    }
    return response

In [39]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [40]:
def get_Google_Search(query: str):
        url = "https://api.tavily.com/search"
        headers = {"Content-Type": "application/json"}
        payload = {
            "api_key":os.getenv('TAVILY_API_KEY') ,
            "query": query,
            "search_depth": "basic",
            "include_answer": True
        }
        response = requests.post(url, headers=headers, json=payload)
        data = response.json()

        answer = data.get("answer", "")
        results = "\n".join(
            [f"- {r['title']} ({r['url']})" for r in data.get("results", [])]
        )
        return f"{answer}\n\nSources:\n{results}"

get_Google_Search("what is the price of wheat today in india")

'As of today, the price of wheat in India is ₹25.82 per kg. The futures price is ₹2,373 per quintal. These prices are current as of 2025-09-05.\n\nSources:\n- Wheat Rate Today In India - Live Market Prices And Trends (https://www.kisandeals.com/mandiprices/WHEAT/ALL/ALL)\n- Wheat Price Today in India - 05 Sep 2025 (https://www.bankbazaar.com/commodity-price/wheat-price.html)\n- Global Wheat Price (https://dir.tridge.com/prices/wheat)\n- Wheat mandi price (https://www.commodityonline.com/mandiprices/wheat)\n- Wheat Futures Price Today - Investing.com India (https://in.investing.com/commodities/ncdex-wheat-futures)'

In [53]:
tools = [{"type": "function", "function": Google_Search}]

In [54]:
# There's a particular dictionary structure that's required to describe our function:

Google_Search = {
    "name": "get_Google_Search",
    "description": "Search Google for real-time information (news, crops, weather, market prices, etc.)",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The search query string",
            },
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [55]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": Google_Search}]

In [56]:
def chat(message, history):
    """Main chat function for Gradio interface"""
    # Convert Gradio history format to OpenAI format
    messages = [{"role": "system", "content": system_prompt}]
    
    for human_msg, ai_msg in history:
        messages.append({"role": "user", "content": human_msg})
        if ai_msg:  # Only add if AI response exists
            messages.append({"role": "assistant", "content": ai_msg})
    
    messages.append({"role": "user", "content": message})
    
    # Make initial API call
    response = openai.chat.completions.create(
        model=model, 
        messages=messages, 
        tools=tools
    )

    # Handle tool calls if present
    if response.choices[0].finish_reason == "tool_calls":
        assistant_message = response.choices[0].message
        tool_response = handle_tool_call(assistant_message)
        
        # Add tool call and response to messages
        messages.append({
            "role": "assistant",
            "content": assistant_message.content,
            "tool_calls": assistant_message.tool_calls
        })
        messages.append(tool_response)
        
        # Make final API call
        final_response = openai.chat.completions.create(
            model=model, 
            messages=messages
        )
        return final_response.choices[0].message.content
    
    return response.choices[0].message.content

In [57]:
# Launch Gradio interface
if __name__ == "__main__":
    interface = gr.ChatInterface(
        fn=chat,
        title="🌾 FarmSakha AI - Smart Crop Advisory Assistant",
        description="Ask me anything about farming, crops, weather, market prices, or agricultural practices in India!",
        examples=[
            "What crops are best for clay soil in Punjab?",
            "Current wheat prices in India",
            "How to control aphids in cotton crop?",
            "Weather forecast for farming this week"
        ]
    )
    interface.launch()

c:\SIH\FarmSakha-AI\venv\Lib\site-packages\gradio\chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.
